In [1]:
import pandas as pd

In [2]:
# Load Raw data with FT sentence vectors
df = pd.read_json('annamayya_dataset_ft_vectors.json')
# Get precomputed MFCC Mean values for each MP3 file
mfcc_data = pd.read_json('mfcc_means.json')

In [3]:
# Load Audio metadata information
mp3 = pd.read_excel('Index.xlsx','Final')
# Unused columns
mp3.drop(['Singer','Album','Song Name','Offset','Unnamed: 6','Unnamed: 7','Unnamed: 8'],axis=1,inplace=True)

In [4]:
# Get feature from raw data based on common key "Samputi"
def get_feature(samputi,d,feature):
    x = d[d['Samputi']==samputi][feature].tolist()
    if len(x) == 1: return x[0]
    return None    

def get_mfcc_mean(fname,d):
    x = d[d['name']==fname]['mfcc_mean'].tolist()
    if len(x) == 1: return x[0]
    return None

In [5]:
mp3['Genre'] = mp3.apply(lambda x:get_feature(x.Samputi,df,'genre'),axis=1)
mp3['Ragam'] = mp3.apply(lambda x:get_feature(x.Samputi,df,'Ragam'),axis=1)
mp3['ft_vector'] = mp3.apply(lambda x:get_feature(x.Samputi,df,'ft_vectors'),axis=1)
mp3.loc[:,'Lyric'] = mp3.apply(lambda x:get_feature(x.Samputi,df,'Lyric'),axis=1)
mp3.loc[:,'mfcc_mean'] = mp3.apply(lambda x:get_mfcc_mean(x['MP3 File'].strip()[:-4],mfcc_data),axis=1)

In [6]:
# Drop entries where there is no mapping from lyrical data
mp3 = mp3[~ mp3['Genre'].isna()]

In [7]:
mp3.shape

(1233, 7)

In [8]:
# Save dataset
mp3.to_json('song_lyric_map.json')